In [1]:
import os
import SimpleITK as sitk
import numpy as np
from utils import *
from convertRaw import *
import gvxrPython3 as gvxr
import json2gvxr
import matplotlib.pyplot as plt

SimpleGVXR 1.0.1 (2021-11-28T22:34:39) [Compiler: Microsoft Visual Studio] on Windows
gVirtualXRay core library (gvxr) 1.1.3 (2021-11-28T22:34:38) [Compiler: Microsoft Visual Studio] on Windows


# Configure gVirtualXRay environment

**Create an OpenGL context and window.**

In [2]:
json2gvxr.initGVXR("notebook-11.json", "OPENGL");

Create an OpenGL context: 512x512


**Create a photon beam.**

In [3]:
json2gvxr.initSourceGeometry()
gvxr.useParallelBeam();

Set up the beam
	Source position: [0, -500, 0, 'um']
	Source shape: ParallelBeam


**Create a detector array.**

In [4]:
json2gvxr.initDetector("notebook-11.json");

Set up the detector
	Detector position: [0, 500, 0, 'um']
	Detector up vector: [0, 0, -1]
	Detector number of pixels: [725, 426]
	Pixel spacing: [0.625, 0.7, 'um']


In [5]:
json2gvxr.initSpectrum(verbose=0)

({100: 1.0}, 'keV', array([100]), array([1.]))

**Create a phantom model from each mesh and its material.**

In [6]:
json2gvxr.initSamples("notebook-11.json", 1);

Load the 3D data

	Load 100HU_tumours in lungman_data/meshes/tumours_100HU.stl using um
	Load -630HU_tumours in lungman_data/meshes/tumours_-630HU.stl using um


OSError: [Errno Invalid transformation:] Translation

**Rotate to achieve the desired orientation**

In [ ]:
#gvxr.rotateScene(90, 0, 0, 1);

**Create a gVirtualXRay projection of the scene**

In [ ]:
xray_image = np.array(gvxr.computeXRayImage());
#xray_image = np.flip(xray_image);

In [ ]:
plt.figure(figsize= (8, 8))
plt.title("xray_image")
plt.imshow(xray_image,cmap=plt.get_cmap("Greys"))
plt.axis("off")

## Plastimatch Parameters
- Normal vector set to 1x,0y,0z or left to right (--nrm)
- Dectector up vector set to 0x,0x,1z (--vup)
- Reconstructed image dimensions 725 by 426 to match gvxr projection (-r)
- Detector size adjusted for spacing $725\times0.625$ by $426\times0.7$ (-z)
- Output type raw (-t)
- Source to detector and source to object set effectively to infinity, mimicking a parellel detector (--sid and --sad)

In [ ]:
os.system('plastimatch drr --nrm "1 0 0" --vup "0 0 1" -r "725 426" -c "580 -57" -z "453.125 298.5" -t raw --sid 32767 --sad 32767 --output Plastimatch_data/lungman_DRR lungman_data/lungman.mha');

In [ ]:
#os.system('python convertRaw.py "Plastimatch_data/lungman_DRR0000.raw" "Plastimatch_data/lungman_DRR.mha" "false" "sitkFloat32" 725 426');

image_DRR = read_raw(binary_file_name="Plastimatch_data/lungman_DRR0000.raw",
                 image_size=[725, 426],
                 sitk_pixel_type=sitk.sitkFloat32,
                 big_endian="false");

image_DRR = np.array(sitk.GetArrayFromImage(image_DRR));

In [ ]:
#----------------------------------------------------REMEMBER TO CHANGE THIS TO TIFF AND USE IMREAD
sitk_reader = sitk.ImageFileReader();
sitk_reader.SetImageIO("MetaImageIO");

sitk_reader.SetFileName("gVirtualXRay_output_data/gvxr_lungman_1-0-0_radiograph_normalised_402.mha");
image_gvxr = np.array(sitk.GetArrayFromImage(sitk_reader.Execute()));

min = np.min(image_DRR);
max = np.max(image_DRR);
image_DRR -= min;
image_DRR /= (max -min);

image_gvxr[image_gvxr < 0.00001] = 0.00001;
image_DRR[image_DRR < 0.00001] = 0.00001;

print(np.min(image_DRR))
print(np.max(image_DRR))
print(np.min(image_gvxr))
print(np.max(image_gvxr))

print(np.mean(image_gvxr));
print(np.mean(image_DRR));


In [ ]:
displayLinearPowerScales(image_gvxr, "gvxr_radiograph", "plots/lungman_gvxr_radiograph")

In [ ]:
displayLinearPowerScales(image_DRR, "DRR", "plots/lungman_DRR")

In [ ]:
fullCompareImages(image_DRR, image_gvxr, "Radiograph comparisons","plots/lungman_full_comparison_radiographs");